# Import dependencies

In [ ]:
!pip install stable-baselines3[extra]

In [ ]:
!apt-get install x11-utils > /dev/null 2>&1
!pip install pyglet > /dev/null 2>&1
!apt-get install -y xvfb python-opengl > /dev/null 2>&1

In [ ]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1

In [ ]:
!pip install --upgrade ipykernel

In [ ]:
import os
#import gym
import gymnasium as gym
import matplotlib.pyplot as plt
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
%matplotlib inline

/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
2023-07-12 17:52:49.188793: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 17:52:50.230529: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Load enviroment

In [ ]:
enviroment_name = "CartPole-v1"
env = gym.make(enviroment_name, render_mode="human")

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
  obs = env.reset()
  done = False
  score = 0

  while not done:
    env.render()
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
    score += reward

  print('Episode: {} Score {}'.format(episode, score))
#env.close()

In [ ]:
env.step(1)

(array([ 0.01988853,  0.22945924,  0.00988458, -0.27532378], dtype=float32),
 1.0,
 False,
 False,
 {})

# Understanding the enviroment

https://www.gymlibrary.dev/

In [ ]:
# 0: Push cart to the left, 1: Push cart to the right
env.action_space

Discrete(2)

In [ ]:
env.action_space.sample()

0

In [ ]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [ ]:
# 0: Cart Position, 1: Cart Velocity, 2: Pole Angle, 3: Pole Angular Velocity
env.observation_space.sample()

array([-3.4021626e+00,  6.3175695e+37, -1.8298085e-01, -1.8386116e+38],
      dtype=float32)

# Train an RL model

* Discrete Single Process: DQN
* Discrete Multi Processed: PPO or A2C
* Continuous Single Process: SAC or TD3
* Continuous Multi Processed: PPO or A2C

In [ ]:
# Make the directories
# Create a folder named Reinforced_learning and inside create another two folders named Logs and Saved models
log_path = os.path.join('Training', 'Logs')

In [ ]:
log_path

'/content/gdrive/MyDrive/Reinforced_learning/Logs'

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


2023-07-12 17:53:50.316611: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-12 17:53:50.354906: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-12 17:53:50.355257: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
env = gym.make(enviroment_name, render_mode="human")
env = DummyVecEnv([lambda: env])

# Policies types: MlpPolicy, CnnPolicy and MultiInputPolicy
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [ ]:
model.learn(total_timesteps=20000)

# Save and reload model

In [ ]:
PPO_path = os.path.join('Training', 'Saved models', 'PPO_Model_Cartpole')

In [ ]:
model.save(PPO_path)

In [ ]:
del model

In [ ]:
model = PPO.load(PPO_path, env=env)

In [ ]:
model.learn(total_timesteps=1000)

# Evaluation

* **ep_len_mean:** on average how long a particular episode lasted before done
* **ep_rew_mean:** the average reward that the agent accumulated per episode

In [ ]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, render=True)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:500.00 +/- 0.00


# Test model

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
  obs = env.reset()
  done = False
  score = 0

  while not done:
    env.render()
    action = model.predict(obs)
    obs, reward, done, truncated, info = env.step(action)
    score += reward

  print('Episode: {} Score {}'.format(episode, score))
#env.close()

# Viewing logs in Tensorboard

* **Average Reward:** Indicates how well the model perform in the particular enviroment.
* **Average Episode Length:** Indicates how long the agent lost in the particular enviroment.

In [ ]:
training_log_path = os.path.join(log_path, 'PPO_2')

In [ ]:
!tensorboard --logdir={training_log_path}

### If not perform very well:


1.   Train for longer
2.   Hyperparameter tuning
3.   Try Different Algorithms



# Adding a callback to the training stage

Can leverage callback functions as part of stable baselines to log out data or save the model under certain conditions.

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [ ]:
save_path = os.path.join('Training', 'Saved models')

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path=save_path,
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# Changing policies

In [ ]:
# New architecture
net_arch = [dict(pi=[128,128,128,128], vf=[128,128,128,128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# Using an alternate algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000)

In [ ]:
model.save()